In [1]:
import pandas as pd 
import numpy as np
import requests
import time
import json
import os   
import sys
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator
from matplotlib import rcParams
from datetime import timedelta
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import concurrent

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
# Configurações iniciais
endarr = ["cat-62", 'bimtra', 'tc-real','tc-prev', 'metaf','metar','esperas','satelite']
base_url = "http://montreal.icea.decea.mil.br:5002/api/v1/"
token = "a779d04f85c4bf6cfa586d30aaec57c44e9b7173"
headers = {'accept': 'application/json'}
directory = ""

session = requests.Session()

def fetch_data(date, endpoint, date_format):
    idate = date.strftime(date_format)
    fdate = (date + timedelta(days=1)).strftime(date_format)
    final_url = f"{base_url}{endpoint}?token={token}&idate={idate}&fdate={fdate}"
    print(f"Fetching data from {final_url}")
    response = session.get(final_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return None

def process_dates(dates, endpoint, date_format, part_num):
    all_data = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_data, date, endpoint, date_format) for date in dates]
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(dates), desc=f"Carregando dados de {endpoint}, Parte {part_num}"):
            data = future.result()
            if data:
                all_data.extend(data)
    # Save data after processing this batch
    save_data(endpoint, all_data, part_num)

def save_data(endpoint, all_data, part_num):
    all_data_df = pd.DataFrame(all_data)
    filename = f"{endpoint}_part_{part_num}.csv"
    all_data_df.to_csv(filename, index=False)
    print(f"Dados de {endpoint} salvos em {filename}")

# Iterar sobre cada endpoint
for endpoint in endarr:
    print(f"Processando {endpoint}")
    total_dates = pd.date_range(start="2022-05-01", end="2023-07-01", freq='D')
    parts = 4
    dates_per_part = len(total_dates) // parts
    extra_dates = len(total_dates) % parts

    if endpoint == "cat-62":
        date_format = '%Y-%m-%d %H:%M:%S.001'
    else:
        date_format = '%Y-%m-%d'

    # Process each part
    for part_num in range(parts):
        start_idx = part_num * dates_per_part
        if part_num == parts - 1:  # Last part includes any extra dates
            end_idx = start_idx + dates_per_part + extra_dates
        else:
            end_idx = start_idx + dates_per_part
        print(f"Processando parte {part_num + 1}/{parts}")
        process_dates(total_dates[start_idx:end_idx], endpoint, date_format, part_num + 1)

Processando cat-62
Processando parte 1/4
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2022-05-01 00:00:00.001&fdate=2022-05-02 00:00:00.001
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2022-05-02 00:00:00.001&fdate=2022-05-03 00:00:00.001
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2022-05-03 00:00:00.001&fdate=2022-05-04 00:00:00.001
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2022-05-04 00:00:00.001&fdate=2022-05-05 00:00:00.001
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2022-05-05 00:00:00.001&fdate=2022-05-06 00:00:00.001
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1

Carregando dados de cat-62, Parte 1:   0%|          | 0/106 [00:00<?, ?it/s]

Carregando dados de cat-62, Parte 1:   1%|          | 1/106 [00:00<00:11,  9.31it/s]

Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2022-05-11 00:00:00.001&fdate=2022-05-12 00:00:00.001
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2022-05-12 00:00:00.001&fdate=2022-05-13 00:00:00.001
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2022-05-13 00:00:00.001&fdate=2022-05-14 00:00:00.001
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2022-05-14 00:00:00.001&fdate=2022-05-15 00:00:00.001
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2022-05-15 00:00:00.001&fdate=2022-05-16 00:00:00.001
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/cat-62?token=a779d04f85c4bf6cfa586d30aae

Carregando dados de cat-62, Parte 1:  25%|██▌       | 27/106 [00:00<00:00, 150.03it/s]

...

Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-05-18&fdate=2023-05-19
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-05-19&fdate=2023-05-20
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-05-20&fdate=2023-05-21
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-05-21&fdate=2023-05-22
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-05-22&fdate=2023-05-23
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-05-23&fdate=2023-05-24
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satel

Carregando dados de satelite, Parte 4:  89%|████████▉ | 97/109 [00:00<00:00, 248.99it/s]

Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-06-20&fdate=2023-06-21
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-06-21&fdate=2023-06-22
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-06-22&fdate=2023-06-23
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-06-23&fdate=2023-06-24
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-06-24&fdate=2023-06-25
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satelite?token=a779d04f85c4bf6cfa586d30aaec57c44e9b7173&idate=2023-06-25&fdate=2023-06-26
Fetching data from http://montreal.icea.decea.mil.br:5002/api/v1/satel

Carregando dados de satelite, Parte 4: 100%|██████████| 109/109 [00:00<00:00, 258.74it/s]

Dados de satelite salvos em satelite_part_4.csv
